In [13]:
# https://pieriantraining.com/tensorflow-lstm-example-a-beginners-guide/
# https://www.tensorflow.org/guide/keras/working_with_rnns

In [14]:
import numpy as np
import pandas as pd
import matplotlib as plt
import tensorflow as tf
import keras
from keras.models import load_model
from keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import joblib

In [15]:
dataset = pd.read_csv('../../data/fraction_preprocessed_data.csv',encoding='ISO-8859-1')
dataset = dataset.drop(['Unnamed: 0'], axis=1)
dataset = dataset.dropna(subset=['Text'])
# dataset = dataset.sample(frac=0.1, random_state=42)
dataset

,Text,Source,Human
0,ING AsiaPacific Companys Problems Research Pap...,Human,1
1,Crisis Love Inquiry Essay Critical Writing fol...,Human,1
2,Sure sex segregation makes lot sense many spor...,Human,1
3,Christianity Islam Values Essay Christianity f...,Human,1
4,Becca liked swim practiced everyday hours ente...,GLM-130B,0
...,...,...,...
78887,Mass Eoghan Chada 10 brother Ruairi 5 said St ...,OPT-30B,0
78888,Asian Teachers Polish Lesson Perfection Stigle...,Human,1
78889,Move knife slowly avoid slipping accidentally ...,OPT-6.7B,0
78890,Good dreams likely occur person feeling relaxe...,Text-Davinci-003,0


In [16]:
# Initialize the tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(dataset['Text'])

In [17]:
# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(dataset['Text'])

# Pad sequences to ensure uniform input length
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [18]:
# unitializing label encoder
label_encoder = LabelEncoder()
label_encoder.fit(dataset['Human'])

# labels to numerical format conversion
encoded_labels = label_encoder.transform(dataset['Human'])

In [19]:
joblib.dump(tokenizer, '../../models/NeuralNetworks/LSTM/Variables/tokenizer.pkl')
joblib.dump(label_encoder, '../../models/NeuralNetworks/LSTM/Variables/label_encoder.pkl')
joblib.dump(encoded_labels, '../../models/NeuralNetworks/LSTM/Variables/encoded_labels.pkl')
joblib.dump(padded_sequences, '../../models/NeuralNetworks/LSTM/Variables/padded_sequences.pkl')

['../../models/NeuralNetworks/LSTM/Variables/padded_sequences.pkl']

In [21]:
# LSTM instantiation
lstm = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    LSTM(128, return_sequences=False),
    Dense(64),
    Dense(32),
    Dense(1, activation='sigmoid')
])

In [22]:
lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
# training phase
lstm.fit(padded_sequences, encoded_labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


1973/1973 ━━━━━━━━━━━━━━━━━━━━ 67s 33ms/step - accuracy: 0.6753 - loss: 0.5914 - val_accuracy: 0.8042 - val_loss: 0.3980
Epoch 2/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 65s 33ms/step - accuracy: 0.8218 - loss: 0.3778 - val_accuracy: 0.8160 - val_loss: 0.3767
Epoch 3/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 96s 49ms/step - accuracy: 0.8572 - loss: 0.3103 - val_accuracy: 0.8429 - val_loss: 0.3452
Epoch 4/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 107s 54ms/step - accuracy: 0.8945 - loss: 0.2450 - val_accuracy: 0.8418 - val_loss: 0.3631
Epoch 5/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 63s 32ms/step - accuracy: 0.9159 - loss: 0.1961 - val_accuracy: 0.8499 - val_loss: 0.3807
Epoch 6/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 61s 31ms/step - accuracy: 0.9383 - loss: 0.1512 - val_accuracy: 0.8456 - val_loss: 0.4234
Epoch 7/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 60s 30ms/step - accuracy: 0.9559 - loss: 0.1085 - val_accuracy: 0.8414 - val_loss: 0.4810
Epoch 8/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 59s 30ms/step - accuracy: 0.9692 - loss: 0.0

In [24]:
# 0.98

In [25]:
lstm.save('../../models/NeuralNetworks/LSTM/lstm_three_98.keras')